In [3]:
# Import packages:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import censusdata
import re
import plotly.figure_factory as ff
import requests

In [58]:
api_key = ""

In [5]:
no_internet = 'B28002_013E'
internet_with_broadband = 'B28002_007E'
# get the margin of error for these variables for next time
#less_than_hs_grad = 'B06009_005E'

white = 'B02001_002E'
black = 'B02001_003E'
native_american = 'B02001_004E'
asian = 'B02001_005E'

median_income = 'B19013_001E'
total_income = 'B19025_001E'

total_population = 'B01003_001E'

race_variables = [white, black, asian, native_american]
race_column_names = ['white', 'black', 'asian', 'native american']

loaction_variables = ['GEO_ID']
location_column_names = ['GEO_ID']

income_variables = [total_income, median_income]
income_column_names = ['total income', 'median_income']

demographics_column_names = race_column_names + ['total population', 'no internet access', 'internet with broadband subscription'] + income_column_names + location_column_names 
variables = race_variables + [total_population] + [no_internet] + [internet_with_broadband] + income_variables + loaction_variables

In [6]:
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

# Import data from EarthPy
data = et.data.get_data('colorado-flood')

# Set working directory to earth-analytics
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [7]:
import json

In [8]:
with open('/Users/benbadnani/Desktop/mass.json') as f:
    state_geo = json.load(f)

In [204]:
mass_data_demographics['% no internet']

 1    0.451098
 2    0.406780
 1    0.392628
 2    0.342513
 3    0.328467
        ...   
 2    0.000000
 4    0.000000
 3    0.000000
 3    0.000000
 1    0.000000
Name: % no internet, Length: 4945, dtype: float64

In [216]:
mass_data_demographics.iloc[0]

white                                              337
black                                               18
asian                                               59
native american                                      0
total population                                   501
no internet access                                 226
internet with broadband subscription                71
total income                                9.3338e+06
median_income                                    17358
GEO_ID                                    250173124001
mean income                                      18630
% no internet                                 0.451098
% internet with broadband subscription        0.141717
zip code                                         01852
Name:  1, dtype: object

In [217]:
mass_data_demographics.iloc[0]['white']/mass_data_demographics.iloc[0]['total population']

0.6726546906187625

In [218]:
pop = mass_data_demographics.iloc[0]

"Demographics: white: {}, black: {}, asian: {}, native american {}, zip code: {}, median income {}".format(pop['white']/pop['total population'],
                                                                                                          pop['black']/pop['total population'],
                                                                                                          pop['asian']/pop['total population'],
                                                                                                          pop['native american']/pop['total population'],
                                                                                                          pop['zip code'],
                                                                                                          pop['median_income'])



'Demographics: white: 0.6726546906187625, black: 0.03592814371257485, asian: 0.11776447105788423, native american 0.0, zip code: 01852, median income 17358'

In [240]:
mass_data_demographics

,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,% no internet,% internet with broadband subscription,zip code
1,337,18,59,0,501,226,71,9333800.0,17358,250173124001,18630.0,0.451098,0.141717,01852
2,275,129,0,0,472,192,189,8967100.0,12841,250277315002,18998.0,0.406780,0.400424,01609
1,842,53,0,0,1248,490,170,19041400.0,15481,250092511001,15258.0,0.392628,0.136218,01841
2,388,38,0,0,581,199,109,11171500.0,15777,250056411012,19228.0,0.342513,0.187608,02722
3,74,265,0,6,411,135,95,6314200.0,9185,250250806013,15363.0,0.328467,0.231144,02119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1806,0,383,0,2228,0,745,164532800.0,195583,250173651002,73848.0,0.000000,0.334381,01776
4,1404,22,28,0,1502,0,700,158521400.0,113713,250250201014,105540.0,0.000000,0.466045,02108
3,1134,57,180,0,1389,0,543,56523200.0,82009,250214180023,40693.0,0.000000,0.390929,02169
3,1314,0,101,0,1500,0,447,141729300.0,250001,250173385003,94486.0,0.000000,0.298000,01890


In [212]:
float(long_lat_codes[1][0])

42.285114

In [222]:
len(long_lat_codes)

4945

In [248]:
import pandas as pd

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[42.4072, -71.3824], zoom_start=8.3)

for step, x in tqdm(enumerate(long_lat_codes), total = 4945): 
    if x: # checking if empty
        if mass_data_demographics.iloc[step]['% no internet'] > .2 and mass_data_demographics.iloc[step]['% internet with broadband subscription'] < .2:
            pop = mass_data_demographics.iloc[step]

            string_ = "Demographics: white: {}, black: {}, asian: {}, native american {}, zip code: {}, median income {}".format(pop['white']/pop['total population'],
                                                                                                          pop['black']/pop['total population'],
                                                                                                          pop['asian']/pop['total population'],
                                                                                                          pop['native american']/pop['total population'],
                                                                                                          pop['zip code'],
                                                                                                          pop['median_income'])
            folium.Marker(
                location=[float(long_lat_codes[step][0]), float(long_lat_codes[step][1])],
                popup=string_,
                icon=folium.Icon(color="purple", icon="info-sign"),
            ).add_to(m)
            
        elif mass_data_demographics.iloc[step]['% no internet'] > .2: 
            
            pop = mass_data_demographics.iloc[step]

            string_ = "Demographics: white: {}, black: {}, asian: {}, native american {}, zip code: {}, median income {}".format(pop['white']/pop['total population'],
                                                                                                          pop['black']/pop['total population'],
                                                                                                          pop['asian']/pop['total population'],
                                                                                                          pop['native american']/pop['total population'],
                                                                                                          pop['zip code'],
                                                                                                          pop['median_income'])
            folium.Marker(
                location=[float(long_lat_codes[step][0]), float(long_lat_codes[step][1])],
                popup=string_,
                icon=folium.Icon(color="red", icon="info-sign"),
            ).add_to(m)
        elif mass_data_demographics.iloc[step]['% internet with broadband subscription'] < .1: 
            
            pop = mass_data_demographics.iloc[step]

            string_ = "Demographics: white: {}, black: {}, asian: {}, native american {}, zip code: {}, median income {}".format(pop['white']/pop['total population'],
                                                                                                          pop['black']/pop['total population'],
                                                                                                          pop['asian']/pop['total population'],
                                                                                                          pop['native american']/pop['total population'],
                                                                                                          pop['zip code'],
                                                                                                          pop['median_income'])
            folium.Marker(
                location=[float(long_lat_codes[step][0]), float(long_lat_codes[step][1])],
                popup=string_,
                icon=folium.Icon(color="blue", icon="info-sign"),
            ).add_to(m)
            
m

  0%|          | 0/4945 [00:00<?, ?it/s]

In [249]:
m.save("more_than_20%_no_access_broadband.html")

In [10]:
mass_block_group_data = censusdata.download(src="acs5",
                                            year=2019, geo=censusdata.censusgeo([("state", '25'),
                                                                                 ("county", "*"),
                                                                                 ("block group", "*")]),
                                            key=api_key, var=variables)

In [11]:
mass_data = pd.DataFrame.copy(mass_block_group_data, deep=True)

mass_data.columns = demographics_column_names
new_indices = []
block_groups = []
tracts = []
c_name = []
for index in mass_data.index.tolist():
    new_index = str(index).split(',')[0].replace("Block Group", "")
    tract = str(index).split(',')[1].replace("Census Tract", "")
    cname = str(str(index).split(',')[2]).strip()
    new_indices.append(new_index)
    tracts.append(tract)
    block_groups.append("Block Group " + new_index)
    c_name.append(cname)

mass_data.index = new_indices
mass_data['mean income'] = round(mass_data['total income'].div(mass_data['total population']))
mass_data['median_income'] = np.where((mass_data.median_income < 0), 'NaN',mass_data.median_income)
mass_data['census tract'] = tracts
mass_data['block group'] = block_groups
mass_data['county'] = c_name
mass_data['% no internet'] = mass_data['no internet access'].div(mass_data['total population'])
mass_data['% internet with broadband subscription'] = mass_data['internet with broadband subscription'].div(mass_data['total population'])
mass_data = mass_data[mass_data['mean income'] > 0]
mass_data = mass_data.dropna().sort_values(['% no internet'], ascending=False)

In [13]:
from tqdm.notebook import tqdm

In [14]:
mass_data = pd.DataFrame.copy(mass_block_group_data, deep=True)

mass_data.columns = demographics_column_names
new_indices = []
block_groups = []
tracts = []
c_name = []
for index in mass_data.index.tolist():
    new_index = str(index).split(',')[0].replace("Block Group", "")
    tract = str(index).split(',')[1].replace("Census Tract", "")
    cname = str(str(index).split(',')[2]).strip()
    new_indices.append(new_index)
    tracts.append(tract)
    block_groups.append("Block Group " + new_index)
    c_name.append(cname)

mass_data.index = new_indices
mass_data['mean income'] = round(mass_data['total income'].div(mass_data['total population']))
mass_data['median_income'] = np.where((mass_data.median_income < 0), 'NaN',mass_data.median_income)
mass_data['census tract'] = tracts
mass_data['block group'] = block_groups
mass_data['county'] = c_name
mass_data['% no internet'] = mass_data['no internet access'].div(mass_data['total population'])
mass_data['% internet with broadband subscription'] = mass_data['internet with broadband subscription'].div(mass_data['total population'])
mass_data = mass_data[mass_data['mean income'] > 0]
mass_data = mass_data.dropna().sort_values(['% no internet'], ascending=False)

In [15]:
#clean GEO_ID
mass_data['GEO_ID'] = mass_data['GEO_ID'].str.replace('1500000US', '')

In [16]:
api_key = ""

In [33]:
import requests
session = requests.Session()

## query for zipcode
def get_zip(geo_id):
    counter.update(1)
    #get census tract and block group number
    try:
        tract = str(geo_id)[:-1]

        url = 'https://www.huduser.gov/hudapi/public/usps?type=6&query=' + tract
        headers = {'Authorization': f'Bearer {api_key}'}
        r = session.get(url, headers=headers, stream = True).json()

        return r['data']['results'][0]['geoid']
    except:
        return 

In [35]:
# get zip codes for each census block
mass_data['zip code'] = mass_data['GEO_ID'].map(lambda a: get_zip(a))

In [34]:
counter = tqdm(total = 4945)

  0%|          | 0/4945 [00:00<?, ?it/s]

In [160]:
new_race_column_names = ['% ' + race for race in race_column_names]

for index in range(len(new_race_column_names)):
    mass_data[new_race_column_names[index]] = round(mass_data[race_column_names[index]].div(mass_data['total population'])*100,1)
mass_data = mass_data.drop(columns = demographics_column_names)
cols = mass_data.columns.tolist()
cols = [cols[6]] + cols[1:6] + cols[7:] + [cols[0]]
mass_data = mass_data[cols]
mass_data['% internet with broadband subscription'] = mass_data['% internet with broadband subscription'].div(.01)
mass_data['% no internet'] = mass_data['% no internet'].div(.01)

In [174]:
county_names = ['Suffolk County', 'Middlesex County', 'Barnstable County', 'Essex County', 'Norfolk County', \
                'Berkshire County', 'Plymouth County', \
                'Bristol County', 'Hampden County', 'Franklin County', 'Worcester County', 'Hampshire County']
county_tables = []

for county in county_names:
    county_info = pd.DataFrame.copy(mass_data[mass_data['county'] == county],deep=True)
    county_tables.append(county_info)

for i,c in enumerate(county_tables):
    c = c.drop(columns='county')
    print(county_names[i], "Demographics and Internet Information")
    display(c[:10])

Suffolk County Demographics and Internet Information


,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,census tract,block group,% no internet,% internet with broadband subscription,zip code
3,74,265,0,6,411,135,95,6314200.0,9185,250250806013,15363.0,806.01,Block Group 3,0.328467,0.231144,None
1,109,480,0,26,639,174,116,14326600.0,NaN,250251404001,22420.0,1404,Block Group 1,0.272300,0.181534,None
1,414,438,25,0,1473,401,280,24442100.0,17391,250250813001,16593.0,813,Block Group 1,0.272234,0.190088,None
2,60,62,324,30,589,144,227,15421500.0,15286,250250705002,26183.0,705,Block Group 2,0.244482,0.385399,None
4,184,170,7,0,453,103,113,15030200.0,38523,250250922004,33179.0,922,Block Group 4,0.227373,0.249448,None
2,54,788,14,0,1007,224,279,36090200.0,35850,250250817002,35839.0,817,Block Group 2,0.222443,0.277061,None
1,90,728,0,0,1241,272,220,21272700.0,22109,250250901001,17142.0,901,Block Group 1,0.219178,0.177276,None
3,153,18,261,0,432,91,101,9773200.0,18542,250250916003,22623.0,916,Block Group 3,0.210648,0.233796,None
1,13,270,0,0,445,91,60,5224100.0,21193,250250817001,11740.0,817,Block Group 1,0.204494,0.134831,None
7,748,51,29,0,1092,211,234,26145400.0,39107,250251701007,23943.0,1701,Block Group 7,0.193223,0.214286,None


Middlesex County Demographics and Internet Information


,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,census tract,block group,% no internet,% internet with broadband subscription,zip code
1,337,18,59,0,501,226,71,9333800.0,17358,250173124001,18630.0,3124,Block Group 1,0.451098,0.141717,None
1,1028,56,182,0,1521,476,242,13188100.0,12058,250173883001,8671.0,3883,Block Group 1,0.312952,0.159106,None
2,392,104,102,0,625,175,115,16221300.0,37447,250173143012,25954.0,3143.01,Block Group 2,0.280000,0.184000,None
2,812,86,264,29,1338,354,322,21887800.0,15250,250173119002,16359.0,3119,Block Group 2,0.264574,0.240658,None
1,867,105,107,0,1080,225,135,16783100.0,26528,250173104001,15540.0,3104,Block Group 1,0.208333,0.125000,None
1,866,281,102,8,1367,262,143,16806500.0,19002,250173101001,12294.0,3101,Block Group 1,0.191661,0.104609,None
1,887,47,18,0,1045,196,319,43024700.0,NaN,250173507001,41172.0,3507,Block Group 1,0.187560,0.305263,None
1,305,21,88,0,504,93,209,21663900.0,NaN,250173515001,42984.0,3515,Block Group 1,0.184524,0.414683,None
6,589,46,121,0,756,139,240,12878400.0,17345,250173418006,17035.0,3418,Block Group 6,0.183862,0.317460,None
1,431,163,447,12,1151,207,237,21412400.0,37448,250173112001,18603.0,3112,Block Group 1,0.179844,0.205908,None


Barnstable County Demographics and Internet Information


...

,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,census tract,block group,% no internet,% internet with broadband subscription,zip code
4,750,0,0,0,766,154,246,32865900.0,54219,250110407014,42906.0,407.01,Block Group 4,0.201044,0.321149,None
1,911,0,0,0,911,177,266,17822400.0,NaN,250110413001,19564.0,413,Block Group 1,0.194292,0.291987,None
3,767,39,7,0,862,123,40,27794200.0,53875,250110404003,32244.0,404,Block Group 3,0.142691,0.046404,None
3,1324,17,0,0,1389,193,439,34327800.0,28692,250110405023,24714.0,405.02,Block Group 3,0.138949,0.316055,None
2,997,99,0,0,1207,160,374,22490700.0,24234,250110407012,18634.0,407.01,Block Group 2,0.132560,0.309859,None
4,729,10,47,0,786,85,307,33900400.0,50924,250110415024,43130.0,415.02,Block Group 4,0.108142,0.390585,None
1,818,0,0,103,921,95,325,28345200.0,57353,250110407021,30777.0,407.02,Block Group 1,0.103149,0.352877,None
3,706,4,0,29,753,76,260,24586200.0,72411,250110405013,32651.0,405.01,Block Group 3,0.100930,0.345286,None
1,711,0,0,2,751,75,178,39421900.0,62639,250110415011,52493.0,415.01,Block Group 1,0.099867,0.237017,None
2,808,6,0,0,814,80,223,29069900.0,80521,250110412002,35712.0,412,Block Group 2,0.098280,0.273956,None


Worcester County Demographics and Internet Information


,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,census tract,block group,% no internet,% internet with broadband subscription,zip code
2,275,129,0,0,472,192,189,8967100.0,12841,250277315002,18998.0,7315,Block Group 2,0.406780,0.400424,None
1,1039,124,125,36,1421,466,346,23012400.0,15262,250277312031,16195.0,7312.03,Block Group 1,0.327938,0.243490,None
1,606,97,0,17,778,166,183,8561000.0,14588,250277314001,11004.0,7314,Block Group 1,0.213368,0.235219,None
5,207,124,0,0,375,80,76,16112300.0,29125,250277326005,42966.0,7326,Block Group 5,0.213333,0.202667,None
1,711,0,95,11,857,179,303,35400100.0,36250,250277322011,41307.0,7322.01,Block Group 1,0.208868,0.353559,None
3,935,233,32,0,1409,292,465,40828500.0,40863,250277316003,28977.0,7316,Block Group 3,0.207239,0.330021,None
6,1113,248,121,16,1714,344,515,23329800.0,12277,250277318006,13611.0,7318,Block Group 6,0.200700,0.300467,None
4,582,489,16,0,1182,233,243,20796700.0,20417,250277304014,17595.0,7304.01,Block Group 4,0.197124,0.205584,None
2,636,50,39,0,819,159,185,14392300.0,27250,250277325002,17573.0,7325,Block Group 2,0.194139,0.225885,None
2,497,84,4,8,756,141,213,15104100.0,18958,250277107002,19979.0,7107,Block Group 2,0.186508,0.281746,None


Hampshire County Demographics and Internet Information


,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,census tract,block group,% no internet,% internet with broadband subscription,zip code
3,1411,58,16,0,1543,216,745,63014400.0,52898,250158219033,40839.0,8219.03,Block Group 3,0.139987,0.482826,None
2,923,0,0,0,923,129,290,22292600.0,37708,250158201022,24152.0,8201.02,Block Group 2,0.139762,0.314193,None
4,802,10,0,0,838,109,304,47263800.0,53958,250158224024,56401.0,8224.02,Block Group 4,0.130072,0.362768,None
5,476,1,0,0,477,62,217,18479200.0,42097,250158211005,38740.0,8211,Block Group 5,0.129979,0.454927,None
2,749,22,0,0,809,92,273,35767300.0,NaN,250158214002,44212.0,8214,Block Group 2,0.113721,0.337454,None
1,672,33,0,0,719,76,215,23927300.0,52059,250158201021,33279.0,8201.02,Block Group 1,0.105702,0.299026,None
2,829,0,5,0,845,89,225,28902600.0,60833,250158227002,34204.0,8227,Block Group 2,0.105325,0.266272,None
1,968,4,0,22,1003,94,289,33804500.0,63011,250158215001,33703.0,8215,Block Group 1,0.093719,0.288136,None
3,1968,55,0,0,2103,180,581,45782000.0,37369,250158201023,21770.0,8201.02,Block Group 3,0.085592,0.276272,None
2,901,58,0,0,971,83,402,31223200.0,41302,250158223002,32156.0,8223,Block Group 2,0.085479,0.414006,None


In [48]:
zip_codes = list(mass_data['zip code'].values)

In [186]:
zip_codes

{}

In [152]:
with open('/Users/benbadnani/Desktop/zip_code.txt') as f:
    lines = f.readlines()

In [153]:
lines = [x.strip('\n') for x in lines]

In [154]:
lines = lines[1:]

In [155]:
lines = [x.strip() for x in lines]

In [178]:
zip_long_lat = []
for x in lines: 
    x,y,z = x.split(',')
    zip_long_lat.append((x,[y,z]))

In [180]:
zip_long_lat = dict(zip_long_lat)

In [184]:
zip_long_lat['01852']

['42.631912', ' -71.296025']

In [188]:
zips = list(mass_data['zip code'].values)

In [190]:
long_lat_codes = []
for x in zips:
    try:
        long_lat_codes.append(zip_long_lat[x])
    except:
        long_lat_codes.append([])
    

In [191]:
long_lat_codes

[['42.631912', ' -71.296025'],
 ['42.285114', ' -71.829987'],
 ['42.712015', ' -71.164873'],
 [],
 ['42.324029', ' -71.085017'],
 ['42.243826', ' -71.843569'],
 ['42.134009', ' -72.565378'],
 ['42.649481', ' -71.348229'],
 ['41.683936', ' -71.177500'],
 ['42.211656', ' -72.642448'],
 ['42.211656', ' -72.642448'],
 ['42.698526', ' -73.084300'],
 ['42.471039', ' -70.941535'],
 ['41.637485', ' -70.938265'],
 ['41.675125', ' -71.148281'],
 ['42.679723', ' -71.300680'],
 [],
 ['42.255083', ' -71.129220'],
 ['42.324029', ' -71.085017'],
 ['42.080692', ' -72.560791'],
 ['41.725050', ' -71.121329'],
 ['41.683936', ' -71.177500'],
 ['42.631912', ' -71.296025'],
 ['41.606252', ' -70.913632'],
 ['41.637485', ' -70.938265'],
 ['41.690243', ' -70.616283'],
 ['42.241557', ' -71.082432'],
 ['42.471039', ' -70.941535'],
 ['42.099958', ' -72.580765'],
 ['41.660995', ' -70.940137'],
 ['42.119674', ' -72.549726'],
 ['42.078371', ' -71.042304'],
 ['42.088907', ' -70.998375'],
 ['42.078371', ' -71.042304']

In [63]:
api = ""

In [192]:
mass_data_demographics = pd.DataFrame.copy(mass_data, deep=True)
mass_data_demographics = mass_data_demographics.drop(columns=['census tract', 'block group','county'])

display(mass_data_demographics)

,white,black,asian,native american,total population,no internet access,internet with broadband subscription,total income,median_income,GEO_ID,mean income,% no internet,% internet with broadband subscription,zip code
1,337,18,59,0,501,226,71,9333800.0,17358,250173124001,18630.0,0.451098,0.141717,01852
2,275,129,0,0,472,192,189,8967100.0,12841,250277315002,18998.0,0.406780,0.400424,01609
1,842,53,0,0,1248,490,170,19041400.0,15481,250092511001,15258.0,0.392628,0.136218,01841
2,388,38,0,0,581,199,109,11171500.0,15777,250056411012,19228.0,0.342513,0.187608,02722
3,74,265,0,6,411,135,95,6314200.0,9185,250250806013,15363.0,0.328467,0.231144,02119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,1806,0,383,0,2228,0,745,164532800.0,195583,250173651002,73848.0,0.000000,0.334381,01776
4,1404,22,28,0,1502,0,700,158521400.0,113713,250250201014,105540.0,0.000000,0.466045,02108
3,1134,57,180,0,1389,0,543,56523200.0,82009,250214180023,40693.0,0.000000,0.390929,02169
3,1314,0,101,0,1500,0,447,141729300.0,250001,250173385003,94486.0,0.000000,0.298000,01890


In [198]:
mass_data_demographics.iloc[0]

white                                              337
black                                               18
asian                                               59
native american                                      0
total population                                   501
no internet access                                 226
internet with broadband subscription                71
total income                                9.3338e+06
median_income                                    17358
GEO_ID                                    250173124001
mean income                                      18630
% no internet                                 0.451098
% internet with broadband subscription        0.141717
zip code                                         01852
Name:  1, dtype: object